In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
 

'apple!' : input_data => apple, output_data = pple!

In [4]:
input_str = 'apple'
label_str = 'pple!'
char_set = sorted(list(set(input_str+label_str)))  #set은 집합이므로 중복문자 허용 x => 문자 5개 남음

print(char_set)

char_set_size =len(char_set)
print(char_set_size)

['!', 'a', 'e', 'l', 'p']
5


In [5]:
input_size = 5
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [6]:
#문자의 벡터값으로 훈련시킴

char_to_index = dict((c, i) for i, c in enumerate(char_set))  #enumerate를 활용하면 char_set의 인덱스를 리턴할 수 있음 

#아래의 코드와 같은 의미
# char_list = [c for c in char_set]
# char_list = []
# for c in char_set:
#     char_set.append(c)

print(char_to_index)


{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [7]:
char_to_index.items()  #key, value를 튜플 형태로 리턴
#cf. char_to_index.values()
#cf. char_to_index.keys()

dict_items([('!', 0), ('a', 1), ('e', 2), ('l', 3), ('p', 4)])

In [8]:
index_to_char ={}
for key, value in char_to_index.items():
    index_to_char[value] = key
    
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [9]:
x_data = [char_to_index[c] for c in input_str]  #apple
y_data = [char_to_index[c] for c in label_str]  #pple!
x_data=[x_data]
y_data=[y_data]

#문자값으로 처리하는 것이 아닌, 문자의 인덱스 값으로 처리함

print(x_data)
print(y_data) 

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [10]:
x_one_hot = [np.eye(char_set_size)[x] for x in x_data]
print(x_one_hot) #데이터의 인덱스의 위치에 1을 넣음

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [11]:
X=torch.FloatTensor(x_one_hot)
Y=torch.LongTensor(y_data)
print(X)
print(Y)

tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.]]])
tensor([[4, 4, 3, 2, 0]])


C:\Users\dhdms\AppData\Local\Temp\ipykernel_18636\3424434832.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  X=torch.FloatTensor(x_one_hot)


In [12]:
print(X.size(), Y.size())

torch.Size([1, 5, 5]) torch.Size([1, 5])


In [13]:
#모델 만들기
#nn.RNN(input_size, hidden_size, output_size) #RNN모델에서 필요한 데이터들 - 만들고 사용해야 하므로 클래스로 만들어서 사용할 것임
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):#모델을 만들 때는 init을 씀
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        #batch_first : 입력데이터 batch의 차원의 위치를 설정
        #(배치크기, 시퀀스 길이, 특징 수)
        #입력의 형태가 (32, 10, 50) 라면 순서대로 위와 같음
        self.fc = nn.Linear(hidden_size, output_size, bias=True)
        
#실제 모델 구현(forward)
    def forward(self, x):
        x, _status = self.rnn(x) #x는 rnn을 수행한 값
        x = self.fc(x)
        return x       
        

In [14]:
#모델 생성
model=Net(input_size, hidden_size, output_size)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=learning_rate)
#최적화 방벙 정함

In [15]:
output=model(X)
print(output)
print(output.size())

tensor([[[ 0.0349,  0.1648, -0.1617, -0.1419, -0.2329],
         [-0.0226,  0.3865, -0.2632, -0.0059, -0.3125],
         [-0.0566,  0.4068, -0.2915, -0.0313, -0.3991],
         [-0.0476,  0.3156,  0.0446, -0.0486, -0.5838],
         [ 0.0076,  0.0928,  0.0844,  0.0511, -0.2798]]],
       grad_fn=<ViewBackward0>)
torch.Size([1, 5, 5])


In [16]:
result = output.data.numpy().argmax(axis = 2)  #0:3차원, 1:열, 2:행
print(np.squeeze(result)) #squeeze는 크기가 1인 차원을 제거함, unsqueeze(dim): dim에 해당하는 위치에 크기가 1인 차원을 추가함 

[1 1 1 1 1]


In [17]:
for i in range(200):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    result = output.data.numpy().argmax(axis=-1) #인덱스로 된 결과
    str_result=''.join([index_to_char[c] for c in np.squeeze(result)]) #문자로 된 결과
    print(i, "loss", loss.item(), "prediction :", result,str_result, 'true:', y_data)

0 loss 1.6892623901367188 prediction : [[1 1 1 1 1]] aaaaa true: [[4, 4, 3, 2, 0]]
1 loss 1.3534389734268188 prediction : [[4 4 4 2 2]] pppee true: [[4, 4, 3, 2, 0]]
2 loss 1.0992273092269897 prediction : [[4 4 4 2 4]] pppep true: [[4, 4, 3, 2, 0]]
3 loss 0.8585538864135742 prediction : [[4 4 3 2 4]] pplep true: [[4, 4, 3, 2, 0]]
4 loss 0.6541686058044434 prediction : [[4 4 3 2 4]] pplep true: [[4, 4, 3, 2, 0]]
5 loss 0.48703402280807495 prediction : [[4 4 3 2 4]] pplep true: [[4, 4, 3, 2, 0]]
6 loss 0.35657793283462524 prediction : [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
7 loss 0.2601460814476013 prediction : [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
8 loss 0.18987752497196198 prediction : [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
9 loss 0.1383204162120819 prediction : [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
10 loss 0.10078851133584976 prediction : [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
11 loss 0.07418277114629745 prediction : [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0

In [18]:
sentence = """Unsqueeze inserts a dimension of size 1 at the specified dim."""

In [19]:
char_set1 = list(set(sentence))
print(char_set1)

['e', 'd', 'u', '.', ' ', 's', 'p', 'm', '1', 'r', 'n', 'a', 'h', '(', 't', 'i', ')', 'f', 'q', 'c', 'z', 'o']


In [20]:
char_dic = {c:i for i, c in enumerate(char_set1)}
char_dic

{'e': 0,
 'd': 1,
 'u': 2,
 '.': 3,
 ' ': 4,
 's': 5,
 'p': 6,
 'm': 7,
 '1': 8,
 'r': 9,
 'n': 10,
 'a': 11,
 'h': 12,
 '(': 13,
 't': 14,
 'i': 15,
 ')': 16,
 'f': 17,
 'q': 18,
 'c': 19,
 'z': 20,
 'o': 21}

In [21]:
#char_dict 크기 구하기
char_dic_size = len(char_dic)
print(char_dic_size)

22


In [22]:
hidden_size = char_dic_size
sequence_length = 10 #데이터가 너무 크면 한 세트에 처리하면 숫자가 너무 커지므로 한 번에 처리할 개수를 정함(훈련데이터 batch size)
learning_rate = 0.1


In [23]:
x_data = []
y_data = []

for i in range(0, len(sentence)-sequence_length):
    x_str = sentence[i:i+sequence_length]
    y_str = sentence[i+1 : i+sequence_length+1]
    print(i, x_str, ':',y_str)
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])    

0 unsqueeze( : nsqueeze(d
1 nsqueeze(d : squeeze(di
2 squeeze(di : queeze(dim
3 queeze(dim : ueeze(dim)
4 ueeze(dim) : eeze(dim) 
5 eeze(dim)  : eze(dim) i
6 eze(dim) i : ze(dim) in
7 ze(dim) in : e(dim) ins
8 e(dim) ins : (dim) inse
9 (dim) inse : dim) inser
10 dim) inser : im) insert
11 im) insert : m) inserts
12 m) inserts : ) inserts 
13 ) inserts  :  inserts a
14  inserts a : inserts a 
15 inserts a  : nserts a d
16 nserts a d : serts a di
17 serts a di : erts a dim
18 erts a dim : rts a dime
19 rts a dime : ts a dimen
20 ts a dimen : s a dimens
21 s a dimens :  a dimensi
22  a dimensi : a dimensio
23 a dimensio :  dimension
24  dimension : dimension 
25 dimension  : imension o
26 imension o : mension of
27 mension of : ension of 
28 ension of  : nsion of s
29 nsion of s : sion of si
30 sion of si : ion of siz
31 ion of siz : on of size
32 on of size : n of size 
33 n of size  :  of size 1
34  of size 1 : of size 1 
35 of size 1  : f size 1 a
36 f size 1 a :  size 1 at
37  size 1 

In [24]:
print(x_data)
print(y_data)

[[2, 10, 5, 18, 2, 0, 0, 20, 0, 13], [10, 5, 18, 2, 0, 0, 20, 0, 13, 1], [5, 18, 2, 0, 0, 20, 0, 13, 1, 15], [18, 2, 0, 0, 20, 0, 13, 1, 15, 7], [2, 0, 0, 20, 0, 13, 1, 15, 7, 16], [0, 0, 20, 0, 13, 1, 15, 7, 16, 4], [0, 20, 0, 13, 1, 15, 7, 16, 4, 15], [20, 0, 13, 1, 15, 7, 16, 4, 15, 10], [0, 13, 1, 15, 7, 16, 4, 15, 10, 5], [13, 1, 15, 7, 16, 4, 15, 10, 5, 0], [1, 15, 7, 16, 4, 15, 10, 5, 0, 9], [15, 7, 16, 4, 15, 10, 5, 0, 9, 14], [7, 16, 4, 15, 10, 5, 0, 9, 14, 5], [16, 4, 15, 10, 5, 0, 9, 14, 5, 4], [4, 15, 10, 5, 0, 9, 14, 5, 4, 11], [15, 10, 5, 0, 9, 14, 5, 4, 11, 4], [10, 5, 0, 9, 14, 5, 4, 11, 4, 1], [5, 0, 9, 14, 5, 4, 11, 4, 1, 15], [0, 9, 14, 5, 4, 11, 4, 1, 15, 7], [9, 14, 5, 4, 11, 4, 1, 15, 7, 0], [14, 5, 4, 11, 4, 1, 15, 7, 0, 10], [5, 4, 11, 4, 1, 15, 7, 0, 10, 5], [4, 11, 4, 1, 15, 7, 0, 10, 5, 15], [11, 4, 1, 15, 7, 0, 10, 5, 15, 21], [4, 1, 15, 7, 0, 10, 5, 15, 21, 10], [1, 15, 7, 0, 10, 5, 15, 21, 10, 4], [15, 7, 0, 10, 5, 15, 21, 10, 4, 21], [7, 0, 10, 5, 15, 21,

In [25]:
x_one_hot = [np.eye(char_dic_size)[x] for x in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print(X)
print(Y)

tensor([[[0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 1., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         ...,
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [26]:
print(X.size(), Y.size())

torch.Size([56, 10, 22]) torch.Size([56, 10])


In [27]:
#모델 만들기
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers,
                        batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)
        
    def forward(self, x):
        x, _status = self.rnn(x)
        x= self.fc(x)
        return x

In [28]:
model = Net(char_dic_size, hidden_size, 2) #층을 2개 쌓겠다.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
outputs = model(X)
print(outputs)

tensor([[[-2.6461e-02, -4.9260e-02, -3.4677e-01,  ..., -2.0541e-01,
          -2.5336e-02, -6.2542e-02],
         [-7.9147e-02, -2.8656e-02, -4.1155e-01,  ..., -2.8943e-01,
          -5.5641e-02, -2.6496e-02],
         [-6.3718e-02, -1.8334e-02, -3.3420e-01,  ..., -3.1300e-01,
          -5.5111e-02, -1.8697e-03],
         ...,
         [-2.5730e-02,  1.8576e-03, -3.4227e-01,  ..., -2.5825e-01,
          -3.7877e-02,  2.0232e-02],
         [-3.4470e-02, -9.2195e-02, -3.6098e-01,  ..., -3.0089e-01,
          -2.3427e-02,  1.1835e-02],
         [-3.9173e-02,  3.2559e-02, -2.7031e-01,  ..., -2.7300e-01,
          -5.3820e-02, -7.1723e-03]],

        [[-2.5971e-02, -2.7644e-02, -3.8409e-01,  ..., -2.1002e-01,
          -5.3283e-03, -2.7915e-02],
         [-5.5836e-02, -1.2144e-02, -3.7001e-01,  ..., -3.3450e-01,
          -4.0564e-02, -2.3446e-02],
         [-4.1748e-03, -1.3446e-02, -3.1421e-01,  ..., -3.1843e-01,
          -8.2912e-02, -2.0390e-02],
         ...,
         [-3.6599e-02, -9

In [30]:
outputs = model(X)
print(outputs[0])

tensor([[-2.6461e-02, -4.9260e-02, -3.4677e-01, -1.6893e-01, -1.1800e-01,
          4.4620e-02,  3.4957e-01, -3.5988e-02,  2.1667e-01, -1.7786e-02,
         -3.0834e-01,  2.3588e-01,  1.3072e-01, -6.8913e-02, -1.4790e-01,
          3.7999e-01, -1.1465e-01, -1.7133e-01, -1.1635e-01, -2.0541e-01,
         -2.5336e-02, -6.2542e-02],
        [-7.9147e-02, -2.8656e-02, -4.1155e-01, -2.1167e-01, -1.2042e-01,
          1.5753e-01,  3.1803e-01,  7.0349e-02,  1.3651e-01,  2.5681e-04,
         -1.7536e-01,  2.3473e-01, -7.9572e-02, -4.5832e-02, -6.1210e-02,
          2.3847e-01, -7.3105e-02, -8.4822e-02,  1.0085e-03, -2.8943e-01,
         -5.5641e-02, -2.6496e-02],
        [-6.3718e-02, -1.8334e-02, -3.3420e-01, -1.8789e-01, -1.2082e-01,
          2.1422e-01,  3.5547e-01,  1.8595e-01,  5.3372e-02, -5.5071e-02,
         -2.5450e-02,  8.4668e-02, -4.4475e-03, -4.4171e-02, -4.8311e-02,
          2.5404e-01, -2.5344e-02, -1.1283e-01,  2.6467e-02, -3.1300e-01,
         -5.5111e-02, -1.8697e-03],
    

In [31]:
# result = outputs.view(-1, char_dic_size)
# print(result.size())

In [32]:
a = ['p', 'y', 't','h','o','n']
a1 = ''.join(a)
a1

'python'

In [33]:
for i in range(200):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs.view(-1, char_dic_size), Y.view(-1))
    
    loss.backward()
    optimizer.step()
    
    results = outputs.argmax(dim = 2)
    predict_str = ""
    
    for j, result in enumerate(results): #10개씩 되어있는 문자들 붙이는 것
        if j == 0 :
            predict_str +=''.join([char_set1[t] for t in result])
        else:
            predict_str += char_set1[result[-1]]
    print(predict_str)

ippappaippppppipppipppppipppppppippippppppipppppppsipisipppippppp
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
ezuaizuuiz1ph 1pezz1ipa z urzzizp z1zuizez1pe zuhzz1peezuhez1pimr
seuiezis eezs . ezezezezezor ezueerezuzzezuemzezs zspeez.imesueze
ietiei e iame i ee ee e e me e  e ie ieme s  ie e   eeee e e ieme
te iee e oime i ee ee i iete s os te i te s  i ti  si ee ime d te
te ate s aim) ateeete a adm) s me oi oime si tn de sne s ipe aime
me a e s a m) afa s s a a m) s ae oi ofze a ans de sfa sfa s aims
me de oi oiz) onzezmenhnzim) siz) dimdnz) 1ffizdne afze1fncd sim)
me de de dime ifeedts d dime side oi sife 1 de  ie iiecifeci sime
eeestr s dims ifser s s dime s deeoi sime 1 dtstie i ecifec) dime
eeeser s aime inserte s aime sioe oi sioe s atsthe i eeifee) dime
ee serze aime ipseres a aime sioi of sioe s atst e speeifee) aime
esndeeze dim) ipsnrts a dime sion of size s atsthe specifies dime
esndeeze dim) dpserts a dime sion of size s at the specified dime
esiueeze(d